In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb
import sys
import os
import warnings
warnings.filterwarnings('ignore')
print(os.getcwd())
code_path = './'
sys.path.append(code_path)
sys.path.append(code_path + 'script')

train_pickle = './cv1_train.pickle'
valid_pickle = './cv1_valid.pickle'
question_file = './questions.csv'
debug = False
validaten_flg = False

train = pd.read_pickle(train_pickle)
valid = pd.read_pickle(valid_pickle)

train = train.loc[train.content_type_id == 0].reset_index(drop=True)
valid = valid.loc[valid.content_type_id == 0].reset_index(drop=True)
# valid = valid.sort_values(['timestamp'])

questions = pd.read_csv('./questions.csv')
map_dict = questions.set_index(['question_id'])['correct_answer'].to_dict()
train['correct_answer'] = train['content_id'].map(map_dict)
valid['correct_answer'] = valid['content_id'].map(map_dict)


sample_size = 4 * 1e7

/root/riiid_comp


In [2]:
from user_feature import initial_user
from state_feature import initial_attempt,dict_trans
from state_feature import dict_trans
from item_feature import initial_item
from utils import initial_dict
from user_feature import valid_user
from state_feature import state_feature
from item_feature import initial_item



ds1 = initial_user(train)
# ds2, state_dict = initial_attempt(train)
# state_dict = dict_trans(state_dict)
user_dict, item_dict = initial_dict(train)
ds3 = initial_item(train, item_dict)

# del ds1, ds2, ds3;gc.collect()

valid_ds1 = valid_user(valid, user_dict)
# valid_ds2, state_dict = state_feature(valid, state_dict)
valid_ds3 = initial_item(valid, item_dict)

del user_dict, item_dict;gc.collect()

initial_user function took 25.310 s
initial_dict function took 6.870 s
initial_item function took 6.250 s
valid_user function took 2.620 s
initial_item function took 0.230 s


12534

In [3]:
prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()
train['prior_question_elapsed_time'] = train.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
train['prior_question_had_explanation'] = train.prior_question_had_explanation.fillna(False).astype('int8')

X = train.iloc[-int(sample_size):]

X[ds1.columns] = ds1.iloc[-int(sample_size):]
# X[ds2.columns] = ds2.iloc[-int(sample_size):]
X[ds3.columns] = ds3.iloc[-int(sample_size):]


add_feature = []
add_feature.extend(list(ds1.columns))
# add_feature.extend(list(ds2.columns))
add_feature.extend(list(ds3.columns))
del ds1, ds3;gc.collect()
# del ds1, ds2, ds3;gc.collect()
# , ds6
# del train;gc.collect()

valid[valid_ds1.columns] = valid_ds1
# valid[valid_ds2.columns] = valid_ds2
valid[valid_ds3.columns] = valid_ds3

valid['prior_question_elapsed_time'] = valid.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
valid['prior_question_had_explanation'] = valid.prior_question_had_explanation.fillna(False).astype('int8')

del valid_ds1, valid_ds3, ;gc.collect()
# valid_ds6

0

In [4]:
from content_encoding import content_emb_dict_init_, initial_embedding_feature

content_emb_dict,content_emb_cols = content_emb_dict_init_(train)
content_emb_ds = initial_embedding_feature(X, content_emb_dict)
valid_content_emb_ds = initial_embedding_feature(valid, content_emb_dict)

X[content_emb_cols] = pd.DataFrame(content_emb_ds, index = X.index)
valid[content_emb_cols] = pd.DataFrame(valid_content_emb_ds, index = valid.index)
add_feature.extend(content_emb_cols)

del content_emb_dict;
del content_emb_ds, valid_content_emb_ds;
gc.collect()

answer_dict_init_ function took 4.990 s
content_encoder1 function took 27.180 s
content_encoder2 function took 10.760 s
read_enc_data function took 0.070 s
content_emb_dict_init_ function took 43.000 s


19703

In [5]:
from feat_group5 import  feats5_wrap, roll_init, valid_feats5_wrap

group5, ds5, col_name = feats5_wrap(X)
roll_group = roll_init(group5, 50)
valid_ds5 = valid_feats5_wrap(valid, roll_group)

X[col_name] = pd.DataFrame(ds5, index=X.index)
valid[col_name] = pd.DataFrame(valid_ds5, index=valid.index)
add_feature.extend(col_name)

del group5;gc.collect()
del ds5,valid_ds5;gc.collect()

  0%|          | 0/177764 [00:00<?, ?it/s]

feat5_group function took 45.030 s


100%|██████████| 177764/177764 [00:30<00:00, 5901.63it/s]


feat5_group function took 1.830 s


0

In [6]:
from rolling_feature import rolling_feat_group, valid_rolling_feature, roll_init, parallel_wrap, resort_array

def resort_array(test_ds, idx):
    new_test = np.zeros(test_ds.shape)
    for i in range(idx.shape[0]):
        new_test[idx[i]] = test_ds[i]
    return new_test

def rolling_feature_wrapper(X, roll_keep):
    shift_period_1 = [1, 5, 10, 20, 30, 40]
    group, roll_idx, name_dict = rolling_feat_group(X, roll_keep)
    roll_ds = parallel_wrap(group, name_dict, shift_period_1)
    roll_ds = resort_array(roll_ds, roll_idx)
    
    rolling_name = []
    func_list = ['mean']
    rolling_name += [f'container_{func}_{p}' for p in shift_period_1 for func in ['mean', 'std']]

    rolling_name += [f'prior_question_elapsed_time_{func}_{p}' for p in shift_period_1 for func in func_list]
    rolling_name += [f'item_mean_{func}_{p}' for p in shift_period_1 for func in func_list]
    rolling_name += [f'task_set_distance_{func}_{p}' for p in shift_period_1 for func in func_list]
    

    return group, name_dict, roll_ds, rolling_name

def valid_rolling_feature_wrapper(valid, group):
    shift_period_1 = [1, 5, 10, 20, 30, 40]
    roll_mean_group = roll_init(group, 300)
    valid_gp, valid_idx, name_dict = rolling_feat_group(valid, roll_keep)
#     valid_gp, valid_shape_list = pre_merge_group(valid_gp, roll_mean_group)
#     valid_roll_ds = valid_parallel_rolling_wrap(valid_gp, valid_shape_list, name_dict, valid_idx, shift_period_1)
    valid_roll_ds = valid_rolling_feature(valid_gp, roll_mean_group, name_dict, shift_period_1)
    valid_roll_ds = resort_array(valid_roll_ds, valid_idx)
    return valid_roll_ds


roll_keep = ['user_id', 'timestamp', 'content_id', 'answered_correctly',
             'prior_question_elapsed_time', 'item_mean', 'task_set_distance', 'part', 'bundle_id']
# shift_period_1 = [1, 5, 10, 20, 30, 40]
group, name_dict, roll_ds, rolling_name = rolling_feature_wrapper(X, roll_keep)
valid_roll_ds = valid_rolling_feature_wrapper(valid, group)
X[rolling_name] = pd.DataFrame(roll_ds, index=X.index)
valid[rolling_name] = pd.DataFrame(valid_roll_ds, index=valid.index)
add_feature.extend(rolling_name)

del group;gc.collect()
del roll_ds,valid_roll_ds;gc.collect()

# # for col in rolling_name:
# #     if col in add_feature:
# #         add_feature.remove(col)

  0%|          | 0/177764 [00:00<?, ?it/s]

rolling_feat_group function took 49.290 s


100%|██████████| 177764/177764 [00:32<00:00, 5514.02it/s]


parallel_wrap function took 35.460 s
rolling_feat_group function took 2.530 s
valid_rolling_feature function took 9.600 s


0

In [7]:
from tqdm import tqdm
from numba import jit,njit
from joblib import Parallel, delayed
from tqdm import tqdm
import gc
from multiprocessing import Process, Manager,Pool
from functools import partial
from numba import prange
import numpy as np
import pandas as pd
from numba import types
from numba.typed import Dict
import functools, time

from multiprocessing import Process, Manager,Pool
from functools import partial
from joblib import Parallel, delayed

def timeit(f):
    def wrap(*args, **kwargs):
        time1 = time.time()
        ret = f(*args, **kwargs)
        time2 = time.time()
        print('{:s} function took {:.3f} s'.format(f.__name__, np.round(time2-time1, 2)))

        return ret
    return wrap

@timeit
def rolling_feat_group(train, col_used):
    a = train[col_used].values
    ind = np.lexsort((a[:,2],a[:,1],a[:,0]))
    a = a[ind]
    g = np.split(a, np.unique(a[:, 0], return_index=True)[1][1:])
    return g, ind, col_used

def resort_array(test_ds, idx):
    new_test = np.zeros(test_ds.shape)
    for i in range(idx.shape[0]):
        new_test[idx[i]] = test_ds[i]
    return new_test

@jit(nopython = True)
def divide_agg(tmp_g, step, name_dict):


    beg = 0
    
    time_idx = name_dict.index('timestamp')
    answer_idx = name_dict.index('answered_correctly')
    item_mean_idx = name_dict.index('item_mean')
    distance_idx = name_dict.index('task_set_distance')
    content_idx = name_dict.index('content_id')
    prior_idx = name_dict.index('prior_question_elapsed_time')
    bundle_idx = name_dict.index('bundle_id')
    
    
    m = 2
    ret1 = np.zeros((tmp_g.shape[0], m))
    ret2 = np.zeros((tmp_g.shape[0], m))
    ret3 = np.zeros((tmp_g.shape[0], m))
    ret4 = np.zeros((tmp_g.shape[0], m))
    ret5 = np.zeros((tmp_g.shape[0], 2))
    ret6 = np.zeros((tmp_g.shape[0], m))
    ret7 = np.zeros((tmp_g.shape[0], m))
    ret8 = np.zeros((tmp_g.shape[0], 4))
    
    ret9 = np.zeros((tmp_g.shape[0], 2))
    ret10 = np.zeros((tmp_g.shape[0], 5))
    
    ret11 = np.zeros((tmp_g.shape[0], 6))
    
    
    bundle_ret1 = np.zeros((tmp_g.shape[0], m))
    bundle_ret2 = np.zeros((tmp_g.shape[0], m))
    bundle_ret3 = np.zeros((tmp_g.shape[0], m))
    bundle_ret4 = np.zeros((tmp_g.shape[0], 3))
    
#     real_time = prior_shift_func(tmp_g[:, time_idx], tmp_g[:, prior_idx])
#     exp_idx = name_dict.index('prior_question_had_explanation')
#     exp_ret1 = np.zeros((tmp_g.shape[0], 3))
    
    for i in step:
        tmp = tmp_g[:beg]
        idx1 = np.where(tmp[:,  answer_idx] == 0)[0]
        if idx1.shape[0] > 0:
#             wrong_time_diff = shift_inner_func(tmp[idx1, time_idx])
            wrong_time_diff = tmp[idx1, time_idx] - np.concatenate((np.full(1, np.nan), tmp[idx1, time_idx][:-1]))
            
            ret1[beg:beg+i, 0] = np.nanmean(tmp[idx1, item_mean_idx])
            ret1[beg:beg+i, 1] = np.nanmedian(tmp[idx1, item_mean_idx])
        
            ret2[beg:beg+i, 0] = np.nanmean(tmp[idx1, distance_idx])
            ret2[beg:beg+i, 1] = np.nanmedian(tmp[idx1, distance_idx])
            
            ret5[beg:beg+i, 0] = tmp_g[beg][time_idx] - tmp[idx1][-1, time_idx]
            ret6[beg:beg+i, 0] = np.nanmean(wrong_time_diff)
            ret6[beg:beg+i, 1] = np.nanmedian(wrong_time_diff)
            

            
            hard_item = np.where(tmp[idx1, item_mean_idx] < 0.6)[0]
            if hard_item.shape[0] > 0:
                hard_time = tmp[idx1,:][hard_item, time_idx]
                ret9[beg:beg+i, 0] = tmp_g[beg, time_idx] - hard_time[-1]
            
        idx2 = np.where(tmp[:, answer_idx] == 1)[0]
        if idx2.shape[0] > 0:
#             right_time_diff = shift_inner_func(tmp[idx2, time_idx])
            right_time_diff = tmp[idx2, time_idx] - np.concatenate((np.full(1, np.nan), tmp[idx2, time_idx][:-1]))
            ret3[beg:beg+i, 0] = np.nanmean(tmp[idx2, item_mean_idx])
            ret3[beg:beg+i, 1] = np.nanmedian(tmp[idx2, item_mean_idx])
            
            ret4[beg:beg+i, 0] = np.nanmean(tmp[idx2, distance_idx])
            ret4[beg:beg+i, 1] = np.nanmedian(tmp[idx2, distance_idx])
            
            ret5[beg:beg+i, 1] = tmp_g[beg][time_idx] - tmp[idx2][-1, time_idx]
        
            ret7[beg:beg+i, 0] = np.nanmean(right_time_diff)
            ret7[beg:beg+i, 1] = np.nanmedian(right_time_diff)
            
            
            hard_item = np.where(tmp[idx2, item_mean_idx] < 0.6)[0]
            if hard_item.shape[0] > 0:
                hard_time = tmp[idx2, :][hard_item, time_idx]
                ret9[beg:beg+i, 1] = tmp_g[beg, time_idx] - hard_time[-1]
               
        for j in range(i):
            last_idx = np.where(tmp[:, content_idx] == tmp_g[beg+j, content_idx])[0]
            if last_idx.shape[0] > 0:
                last_content_time = tmp[last_idx[-1], time_idx]
                ret8[beg+j, 1] = np.nanmean(tmp[last_idx, answer_idx])
                ret8[beg+j, 2] = np.nansum(tmp[last_idx, answer_idx])
                ret8[beg+j, 3] = tmp[last_idx, answer_idx].shape[0]
            else:
                last_content_time = np.nan
            ret8[beg+j, 0] = tmp_g[beg+j, time_idx] - last_content_time
            
        ret11[beg:beg+i, 0] = ret2[beg:beg+i, 1]/(1e-6 + ret4[beg:beg+i, 1])
        ret11[beg:beg+i, 1] = ret1[beg:beg+i, 1]/(1e-6 + ret3[beg:beg+i, 1])
        ret11[beg:beg+i, 2] = ret6[beg:beg+i, 1]/(1e-6 + ret7[beg:beg+i, 1])

        ret11[beg:beg+i, 3] = ret5[beg:beg+i, 0]/(1e-6 + ret6[beg:beg+i, 1])
        ret11[beg:beg+i, 4] = ret5[beg:beg+i, 0]/(1e-6 + ret5[beg:beg+i, 1])
        ret11[beg:beg+i, 5] = ret5[beg:beg+i, 1]/(1e-6 + ret7[beg:beg+i, 1])
        

            
        for shift in range(1, 6):
            if tmp.shape[0] > shift:
                ret10[beg:beg+i, shift - 1] = tmp[-shift, distance_idx]
        
        selected_bundle = tmp_g[beg, bundle_idx]
        group_idx = np.where(tmp[:, bundle_idx] == selected_bundle)[0]
        if group_idx.shape[0] > 0:
            part_diff_time = tmp[group_idx, time_idx] - np.concatenate((np.full(1, np.nan), tmp[group_idx, time_idx][:-1]))
            bundle_ret1[beg:beg+i, 0] = np.nanmean(tmp[group_idx, item_mean_idx])
            bundle_ret1[beg:beg+i, 1] = np.nanmedian(tmp[group_idx,  item_mean_idx])
            bundle_ret2[beg:beg+i, 0] = np.nanmean(tmp[group_idx, distance_idx])
            bundle_ret2[beg:beg+i, 1] = np.nanmedian(tmp[group_idx, distance_idx])
            bundle_ret3[beg:beg+i, 0] = np.nanmean(part_diff_time)
            bundle_ret3[beg:beg+i, 1] = np.nanmedian(part_diff_time)
        
            bundle_ret4[beg:beg+i, 0] = np.nansum(tmp[group_idx, answer_idx])
            bundle_ret4[beg:beg+i, 1] = tmp[group_idx, answer_idx].shape[0]
            bundle_ret4[beg:beg+i, 2] = np.nanmean(tmp[group_idx, answer_idx])
            
#         selected_exp = tmp_g[beg, exp_idx]
#         group_idx = np.where(tmp[:, exp_idx] == selected_exp)[0]
#         if group_idx.shape[0] > 0:
        
#             exp_ret1[beg:beg+i, 0] = np.nansum(tmp[group_idx, answer_idx])
#             exp_ret1[beg:beg+i, 1] = tmp[group_idx, answer_idx].shape[0]
#             exp_ret1[beg:beg+i, 2] = np.nanmean(tmp[group_idx, answer_idx])
            
        beg += i
    
    bundle_ret = np.concatenate((bundle_ret1, bundle_ret2, bundle_ret3, bundle_ret4), axis = 1)
    ret = np.concatenate((ret1, ret2, ret3, ret4, ret5, ret6, ret7, ret8, ret9, 
                          ret10, bundle_ret, ret11), axis = 1)
    return ret


            
            
@jit(nopython = True)
def last_group_cal(tmp_g, step, name_dict):
    beg = 0
    part_group = np.zeros((tmp_g.shape[0], 9))
    
    part_idx = name_dict.index('part')
    time_idx = name_dict.index('timestamp')
    answer_idx = name_dict.index('answered_correctly')
    item_mean_idx = name_dict.index('item_mean')
#     distance_idx = name_dict.index('task_set_distance')
    
    for i in step:
        tmp = tmp_g[:beg]
        selected_part = tmp_g[beg, part_idx]
        idx_list = []
        data = np.where(tmp[:, part_idx] == selected_part)[0]
        idx = np.where(np.diff(data) != 1)[0] + 1
        if idx.shape[0] > 0:
            for cnt in range(idx.shape[0] + 1):
                if cnt == 0:
                    idx_list.append(data[np.arange(idx[cnt])])
                elif cnt == idx.shape[0]:
                    idx_list.append(data[idx[cnt - 1]:])
                else:
                    idx_list.append(data[np.arange(idx[cnt - 1], idx[cnt])])
        else:
            if data.shape[0] > 0:
                idx_list.append(data)
                
        if len(idx_list) > 0:
            last_group = idx_list[-1]
        else:
            last_group = np.full((0, ), np.nan, dtype=np.int64)
            
        if last_group.shape[0] > 0:
            time_begin = tmp[last_group[0], time_idx]
            time_end = tmp[last_group[-1], time_idx]
            answer_ratio = np.mean(tmp[last_group, answer_idx])
            answer_sum = np.sum(tmp[last_group, answer_idx])
            answer_count = tmp[last_group, answer_idx].shape[0]
            part_time_diff = \
            tmp[last_group, time_idx] - np.concatenate((np.full(1, np.nan), tmp[last_group, time_idx][:-1]))
            item_mean_group = tmp[last_group, item_mean_idx]
        else:
            time_begin = np.nan
            time_end = np.nan
            answer_ratio = np.nan
            answer_sum = np.nan
            answer_count = np.nan
            part_time_diff = np.full((1, ), np.nan)
            item_mean_group = np.full((1, ), np.nan)
#             task_set_group = np.full((1, ), np.nan)
            
        begin_diff = tmp_g[beg, time_idx] - time_begin
        end_diff = tmp_g[beg, time_idx] - time_end
        time_last = time_end - time_begin
        time_diff_mean = np.nanmean(part_time_diff)
        time_diff_median = np.nanmedian(part_time_diff)
    
        part_group[beg:beg + i, 0] = begin_diff
        part_group[beg:beg + i, 1] = end_diff
        part_group[beg:beg + i, 2] = time_last
        part_group[beg:beg + i, 3] = time_diff_mean
        part_group[beg:beg + i, 4] = answer_ratio
        part_group[beg:beg + i, 5] = answer_sum
        part_group[beg:beg + i, 6] = answer_count
        part_group[beg:beg + i, 7] = np.mean(item_mean_group)
        part_group[beg:beg + i, 8] = np.median(item_mean_group)
        


        
        beg += i
    return part_group

# np.full((tmp_g.shape[0], m), np.nan)
@jit(nopython = True)
def full_group_cal(tmp_g, step, name_dict):
    beg = 0
    m = 2
    ret1 = np.zeros((tmp_g.shape[0], m))
    ret2 = np.zeros((tmp_g.shape[0], m))
    ret3 = np.zeros((tmp_g.shape[0], m))
    
    ret4 = np.zeros((tmp_g.shape[0], m))
    ret6 = np.zeros((tmp_g.shape[0], m))
    ret7 = np.zeros((tmp_g.shape[0], m))
    
    ret8 = np.zeros((tmp_g.shape[0], 7))
    
    ret9 = np.zeros((tmp_g.shape[0], 6 + 3 + 2))
#     ret10 = np.zeros((tmp_g.shape[0], 2 + 2))

#     ret10 = np.full((tmp_g.shape[0], 7), np.nan)
    part_idx = name_dict.index('part')
    time_idx = name_dict.index('timestamp')
    item_mean_idx = name_dict.index('item_mean')
    distance_idx = name_dict.index('task_set_distance')
    answer_idx = name_dict.index('answered_correctly')
    content_idx = name_dict.index('content_id')
    for i in step:
        tmp = tmp_g[:beg]
        selected_part = tmp_g[beg, part_idx]
        group_idx = np.where(tmp[:, part_idx] == selected_part)[0]
        if group_idx.shape[0] > 0:
            tmp_group = tmp[group_idx]
            idx1 = np.where(tmp_group[:,  answer_idx] == 0)[0]
            if idx1.shape[0] > 0:
                wrong_time_diff = tmp_group[idx1, time_idx] - np.concatenate((np.full(1, np.nan), 
                                                                              tmp_group[idx1, time_idx][:-1]))
                ret1[beg:beg+i, 0] = np.nanmean(tmp_group[idx1, item_mean_idx])
                ret1[beg:beg+i, 1] = np.nanmedian(tmp_group[idx1, item_mean_idx])
        
                ret2[beg:beg+i, 0] = np.nanmean(tmp_group[idx1, distance_idx])
                ret2[beg:beg+i, 1] = np.nanmedian(tmp_group[idx1, distance_idx])
            
                ret6[beg:beg+i, 0] = np.nanmean(wrong_time_diff)
                ret6[beg:beg+i, 1] = np.nanmedian(wrong_time_diff)

                ret9[beg:beg+i, 9] = \
                (tmp_g[beg, time_idx] - tmp[idx1[-1], time_idx])/(1e-6 + ret6[beg, 1])
                
            idx2 = np.where(tmp_group[:, answer_idx] == 1)[0]
            if idx2.shape[0] > 0:
                right_time_diff = tmp_group[idx2, time_idx] - np.concatenate((np.full(1, np.nan),
                                                                              tmp_group[idx2, time_idx][:-1]))
                ret3[beg:beg+i, 0] = np.nanmean(tmp_group[idx2, item_mean_idx])
                ret3[beg:beg+i, 1] = np.nanmedian(tmp_group[idx2, item_mean_idx])
            
                ret4[beg:beg+i, 0] = np.nanmean(tmp_group[idx2, distance_idx])
                ret4[beg:beg+i, 1] = np.nanmedian(tmp_group[idx2, distance_idx])
            
                ret7[beg:beg+i, 0] = np.nanmean(right_time_diff)
                ret7[beg:beg+i, 1] = np.nanmedian(right_time_diff)
                
                ret9[beg:beg+i, 10] = \
                (tmp_g[beg, time_idx] - tmp[idx2[-1], time_idx])/(1e-6 + ret7[beg, 1])
                


            ret8[beg:beg+i, 0] = np.nansum(tmp_group[:, answer_idx])
            ret8[beg:beg+i, 1] = tmp_group[:, answer_idx].shape[0]
            ret8[beg:beg+i, 2] = np.nanmean(tmp_group[:, answer_idx])
            ret8[beg:beg+i, 3] = ret8[beg:beg+i, 1]/tmp.shape[0]
            ret8[beg:beg+i, 4] = ret8[beg:beg+i, 0]/tmp.shape[0]
            
            

            ret8[beg:beg+i, 5] = \
            (tmp_g[beg, time_idx] - tmp[group_idx[-1], time_idx])/(1e-6 + np.nanmedian(wrong_time_diff))
            ret8[beg:beg+i, 6] = \
            (tmp_g[beg, time_idx] - tmp[group_idx[-1], time_idx])/(1e-6 + np.nanmedian(right_time_diff))
            

            
            ret9[beg:beg+i, 0] = np.nansum(tmp_group[-1:, answer_idx])
            ret9[beg:beg+i, 1] = np.nanmean(tmp_group[-1:, answer_idx])
            
            ret9[beg:beg+i, 2] = np.nansum(tmp_group[-5:, answer_idx])
            ret9[beg:beg+i, 3] = np.nanmean(tmp_group[-5:, answer_idx])
            
            ret9[beg:beg+i, 4] = np.nansum(tmp_group[-10:, answer_idx])
            ret9[beg:beg+i, 5] = np.nanmean(tmp_group[-10:, answer_idx])

            short_time_diff = tmp_group[-5:, time_idx] - np.concatenate((np.full(1, np.nan),
                                                       tmp_group[-5:, time_idx][:-1]))
            ret9[beg:beg+i, 6] = \
            (tmp_g[beg, time_idx] - tmp[group_idx[-1], time_idx])/(1e-6 + np.nanmedian(short_time_diff))
            
            short_time_diff = tmp_group[-10:, time_idx] - np.concatenate((np.full(1, np.nan),
                                                       tmp_group[-10:, time_idx][:-1]))
            ret9[beg:beg+i, 7] = \
            (tmp_g[beg, time_idx] - tmp[group_idx[-1], time_idx])/(1e-6 + np.nanmedian(short_time_diff))
            

            short_time_diff = tmp_group[-20:, time_idx] - np.concatenate((np.full(1, np.nan),
                                                       tmp_group[-20:, time_idx][:-1]))
            ret9[beg:beg+i, 8] = \
            (tmp_g[beg, time_idx] - tmp[group_idx[-1], time_idx])/(1e-6 + np.nanmedian(short_time_diff))
            
#             ret9[beg:beg+i, 10] = np.nanmean(short_time_diff)
#             ret9[beg:beg+i, 11] = np.nanmedian(short_time_diff)
            
        beg += i
#     ret = np.concatenate((ret1, ret2, ret3, ret4, ret8, ret9), axis = 1)
    ret = np.concatenate((ret1, ret2, ret6, ret3, ret4, ret7, ret8, ret9), axis = 1)
#     ret = np.concatenate((ret1, ret2, ret3), axis = 1)
    return ret


In [8]:
@jit(nopython = True)
def user_answer_cal(tmp_g, step, name_dict, item_answer):
    beg = 0
    
    content_idx = name_dict.index('content_id')
    time_idx = name_dict.index('timestamp')
    item_mean_idx = name_dict.index('item_mean')
    answer_idx = name_dict.index('answered_correctly')
    user_answer_idx = name_dict.index('user_answer')
    distance_idx = name_dict.index('task_set_distance')
    item_answer_idx = name_dict.index('correct_answer')
    part_idx = name_dict.index('part')
    
    ans = np.zeros((tmp_g.shape[0], 1),)
    tmp_ans = item_answer[tmp_g[:,content_idx].astype(np.uint16), :]
    for k in range(tmp_ans.shape[0]):
        ans[k] = tmp_ans[k, np.uint8(tmp_g[k, user_answer_idx])]
        
#     m = 
    ret1 =  np.zeros((tmp_g.shape[0], 2))
    ret2 = np.zeros((tmp_g.shape[0], 3 * 2))
    ret3 = np.zeros((tmp_g.shape[0], 2 * 2))
    ret4 = np.zeros((tmp_g.shape[0], 4 + 2))
    
    tmp_ = np.concatenate((tmp_g, ans), axis = 1)
    for i in step:
        tmp = tmp_[:beg]         
            
        if tmp.shape[0] > 0:
            ret1[beg:beg+i, 0] = np.mean(tmp[:, -1])
            ret1[beg:beg+i, 1] = np.median(tmp[:, -1])
            for j in range(i):
                item_ans = tmp_g[beg + j, item_answer_idx]
                ans_index = np.where(tmp[:, user_answer_idx] == item_ans)[0]
                if ans_index.shape[0] > 0:
                    ret2[beg+j, 0] = np.sum(tmp[ans_index, answer_idx])
                    ret2[beg+j, 1] = np.mean(tmp[ans_index, answer_idx])
                    ret2[beg+j, 2] = tmp[ans_index, answer_idx].shape[0]
                    
                    ret4[beg+j, 4] = tmp_g[beg+j, time_idx] - tmp[ans_index[-1], time_idx]
                    
                ans_index = np.where(tmp[:, item_answer_idx] == item_ans)[0]
                if ans_index.shape[0] > 0:
                    ret2[beg+j, 3] = np.sum(tmp[ans_index, answer_idx])
                    ret2[beg+j, 4] = np.mean(tmp[ans_index, answer_idx])
                    ret2[beg+j, 5] = tmp[ans_index, answer_idx].shape[0]
                    
                    ret4[beg+j, 5] = tmp_g[beg+j, time_idx] - tmp[ans_index[-1], time_idx]
        
                    
            idx1 = np.where(tmp[:,  answer_idx] == 0)[0]
            if idx1.shape[0] > 0:
                y = np.bincount(tmp[idx1, user_answer_idx].astype(np.uint8))
                ii = np.nonzero(y)[0]
                tmp_y = np.vstack((ii, y[ii])).T
                
                ret3[beg:beg+i, 0] = tmp_y[:,0][np.argmax(tmp_y[:,1])]
                ret3[beg:beg+i, 1] = tmp_y[:,1][np.argmax(tmp_y[:,1])]
                

            idx2 = np.where(tmp[:, answer_idx] == 1)[0]
            if idx2.shape[0] > 0:
                
                y = np.bincount(tmp[idx2, user_answer_idx].astype(np.uint8))
                ii = np.nonzero(y)[0]
                tmp_y = np.vstack((ii, y[ii])).T
                
                ret3[beg:beg+i, 2] = tmp_y[:,0][np.argmax(tmp_y[:,1])]
                ret3[beg:beg+i, 3] = tmp_y[:,1][np.argmax(tmp_y[:,1])]

           
            tmp1 = tmp[-20:,:]
            y = np.bincount(tmp1[:,user_answer_idx].astype(np.uint8))
            ii = np.nonzero(y)[0]
            tmp_y = np.vstack((ii, y[ii])).T 
            ret4[beg:beg+i, 0] = tmp_y[:,0][np.argmax(tmp_y[:,1])]
            ret4[beg:beg+i, 1] = tmp_y[:,1][np.argmax(tmp_y[:,1])]
            
            idx0 = np.where(tmp1[:, user_answer_idx] == np.argmax(tmp_y[:,1]))[0]
            ret4[beg:beg+i, 2] = np.nanmean(tmp1[idx0, answer_idx])
            ret4[beg:beg+i, 3] = np.nansum(tmp1[idx0, answer_idx])
            

#             y = np.bincount(tmp1[:, part_idx].astype(np.uint8))
#             ii = np.nonzero(y)[0]
#             tmp_y = np.vstack((ii, y[ii])).T 
#             ret4[beg:beg+i, 4] = ii.shape[0]
#             ret4[beg:beg+i, 5] = np.mean(tmp_y[:,1])
            
        beg += i
    ret = np.concatenate((ret1, ret2, ret3, ret4), axis = 1)
    return ret

In [9]:
def last_group_cal_wrap(tmp_g, name_dict, item_answer_dict):
    step = np.unique(tmp_g[:, 1], return_counts=True)[1]
    ds1 = last_group_cal(tmp_g, step, name_dict)
    ds2 = full_group_cal(tmp_g, step, name_dict)
    ds3 = divide_agg(tmp_g, step, name_dict)
    ds4 = user_answer_cal(tmp_g, step, name_dict, item_answer_dict)
    ds = np.concatenate((ds1, ds2, ds3, ds4), axis = 1)
    return ds

from multiprocessing import Process, Manager,Pool
from functools import partial
from joblib import Parallel, delayed

@timeit
def parallel_last_group_wrap(group, name_dict, last_idx, item_answer_dict):
    res = Parallel(n_jobs = 25, backend = 'loky')\
              (delayed(last_group_cal_wrap)(group[i], name_dict, item_answer_dict)
              for i in tqdm(range(len(group))))
    ans = np.concatenate(res)
    ans = resort_array(ans, last_idx)
    return ans


@timeit
def last_group_feature(g, name_dict, idx, item_answer_dict):
    res = []
    for i in tqdm(range(len(g))):
        tmp_g = g[i]
        tmp_res = last_group_cal_wrap(tmp_g, name_dict, item_answer_dict)
        res.append(tmp_res)
    ans = np.concatenate(res)
    ans = resort_array(ans, idx)
    return ans

@timeit
def valid_last_group_feature(valid_gp, rolling_gp, name_dict, valid_idx, item_answer_dict):
    res = []
    for i in range(len(valid_gp)):
        tmp_g = valid_gp[i]
        valid_shape = tmp_g.shape[0]
        if tmp_g[0,0] in rolling_gp:
            tmp_rolling = rolling_gp[tmp_g[0, 0]]
            tmp_g = np.concatenate([tmp_rolling, tmp_g])
        
        tmp_res = last_group_cal_wrap(tmp_g, name_dict, item_answer_dict)
        tmp_res = tmp_res[-valid_shape:]
        res.append(tmp_res)
    ans = np.concatenate(res)
    ans = resort_array(ans, valid_idx)
    return ans

def pre_merge_group(valid_gp, rolling_gp):
    valid_shape_list = []
    for i in range(len(valid_gp)):
        tmp_g = valid_gp[i]
        valid_shape = tmp_g.shape[0]
        if tmp_g[0,0] in rolling_gp:
            tmp_rolling = rolling_gp[tmp_g[0, 0]]
            tmp_g = np.concatenate([tmp_rolling, tmp_g])
        valid_gp[i] = tmp_g
        
        valid_shape_list.append(valid_shape)
    return valid_gp, valid_shape_list


def valid_last_group_wrap(tmp_g, valid_shape, name_dict, item_answer_dict):
    tmp_res = last_group_cal_wrap(tmp_g, name_dict, item_answer_dict)
    tmp_res = tmp_res[-valid_shape:]
    return tmp_res

@timeit
def valid_parallel_last_group_wrap(group, valid_shape_list, name_dict, valid_idx, item_answer_dict):
    res = Parallel(n_jobs = 25, backend = 'loky')\
              (delayed(valid_last_group_wrap)(group[i], valid_shape_list[i], name_dict, item_answer_dict)
              for i in tqdm(range(len(group))))
    ans = np.concatenate(res)
    ans = resort_array(ans, valid_idx)
    return ans


In [10]:


last_group_name = []
last_group_name += ['begin_time_diff', 'end_time_diff', 'part_duration_time', 'part_time_diff_mean']
last_group_name += ['part_session_mean', 'part_session_sum', 'part_session_count']
last_group_name += ['last_part_item_mean', 'last_part_item_median']
# last_group_name += ['part_time_diff_median']
# last_group_name += ['session_count_mean', 'session_count_std', 'session_total_count']

# last_group_name += ['last_task_set_distance_mean', 'last_task_set_distance_median']

# last_group_name += [f'full_group_{var}_{func}' 
#  for var in ['item_mean', 'task_set_distance', 'timestamp'] for func in ['mean', 'median']]
last_group_name += [f'full_group{g}_{var}_{func}'  for g in [0, 1]
 for var in ['item_mean', 'task_set_distance', 'timestamp'] for func in ['mean', 'median']]
last_group_name += ['part_sum', 'part_count', 'part_mean']
last_group_name += ['part_count_global_ratio', 'part_sum_global_ratio']
last_group_name += ['part_time_wrong_div', 'part_time_right_div']
# last_group_name += ['last_part_time_wrong_div', 'last_part_time_right_div']

# last_group_name += ['part_duration_answer_mean', 'part_interval_answer_mean']

last_group_name += [f'part_{func}_{i}' for i in [1, 5, 10] for func in ['sum', 'mean']]
last_group_name += ['last_5_part_time_div', 'last_10_part_time_div', 
                    'last_20_part_time_div']
last_group_name += ['last_right_time_diff', 'last_wrong_time_diff']
# last_group_name += ['last_part_item', 'last_part_task_set_distance',
#                     'last_part_task_item_mean', 'last_part_time_diff']
# last_group_name += ['item_mean_right_div', 'item_mean_wrong_div']

new_func_list = ['mean', 'median']
# , 'min', 'max', 'median'
last_group_name += [f"cum_answer0_{func}_{val}" 
                 for val in ['item_mean', 'task_set_distance'] 
                 for func in new_func_list]
last_group_name += [f"cum_answer1_{func}_{val}" 
                 for val in ['item_mean', 'task_set_distance'] 
                 for func in new_func_list]
last_group_name += ["cum_answer0_time_diff", "cum_answer1_time_diff"]
last_group_name += [f"global_task_set_shift{i}" for i in range(1, 6)]

last_group_name += [f"cum_answer0_{func}_{val}" 
                 for val in ['wrong_time_diff'] 
                 for func in new_func_list]
last_group_name += [f"cum_answer1_{func}_{val}" 
                 for val in ['right_time_diff'] 
                 for func in new_func_list]

last_group_name += ['last_content_id_time_diff']
last_group_name += ['content_correct_mean', 'content_correct_sum', 'content_correct_count']
last_group_name += ['hard_answer0_time', 'hard_answer1_time']

last_group_name += [f'full_bundle_{var}_{func}' 
 for var in ['item_mean', 'task_set_distance', 'timestamp'] for func in ['mean', 'median']]
last_group_name += ['bundle_sum', 'bundle_mean', 'bundle_count']
last_group_name += ['diff_lag_median_div', 'diff_item_median_div', 'diff_time_median_div']
last_group_name += ['diff_item_mean_div', 'diff_task_set_mean_div', 'diff_timestamp_mean_div']
# last_group_name += ['wrong_time_diff20', 'wrong_time_diff10', 'right_time_diff20', 'right_time_diff10']


last_group_name += [f'user_trend_{func}' for func in ['mean', 'median']]
last_group_name += [f'user_trend_roll_{p}_{func}' 
                    for p in ['user_ans', 'item_ans'] 
                    for func in ['sum', 'mean', 'count']]
last_group_name += [f'new_Feat{i}' for i in range(4)]
# last_group_name += [f'part_new_Feat{i}' for i in range(4)]

last_group_name += ['last_20_frequent_answer', 'last_20_frequent_answer_count',
                    'last_20_frequent_answer_mean', 'last_20_frequent_answer_sum']
last_group_name += ['last_user_same_answer_tf', 'last_item_same_answer_tf']

item_answer_dict = train.groupby(['content_id', 'user_answer'])['user_answer'].agg('count').unstack()
item_answer_dict.fillna(0, inplace = True)
item_answer_dict = item_answer_dict.to_numpy()
total_sum = np.sum(item_answer_dict, axis = 1)
for i in range(4):
    item_answer_dict[:, i] = item_answer_dict[:, i]/total_sum
    
roll_keep = ['user_id', 'timestamp', 'content_id', 'answered_correctly',
             'prior_question_elapsed_time', 'item_mean', 'task_set_distance', 'part', 'bundle_id', 
             'user_answer', 'correct_answer']
group, last_idx, name_dict = rolling_feat_group(X, roll_keep)
last_group_ds = parallel_last_group_wrap(group, name_dict, last_idx, item_answer_dict)

last_part_group = roll_init(group, 20000)
# last_part_group = np.load('./local_whole_group_data_v4.npy', allow_pickle = True).item()

valid_gp, valid_idx, name_dict = rolling_feat_group(valid, roll_keep)
valid_gp, valid_shape_list = pre_merge_group(valid_gp, last_part_group)
valid_last_group_ds = valid_parallel_last_group_wrap(valid_gp, valid_shape_list, name_dict, 
                                                     valid_idx, item_answer_dict)

X[last_group_name] = pd.DataFrame(last_group_ds, index=X.index)
valid[last_group_name] = pd.DataFrame(valid_last_group_ds, index=valid.index)
add_feature.extend(last_group_name)

del group, valid_gp, last_part_group;
del last_group_ds, valid_last_group_ds;
gc.collect()


# for col in last_group_name:
#     add_feature.remove(col)
# X.drop(last_group_name, axis = 1, inplace = True)
# valid.drop(last_group_name, axis = 1, inplace = True)


  0%|          | 0/177764 [00:00<?, ?it/s]

rolling_feat_group function took 57.620 s


100%|██████████| 177764/177764 [04:56<00:00, 599.21it/s] 


parallel_last_group_wrap function took 368.460 s
rolling_feat_group function took 2.770 s


100%|██████████| 25298/25298 [00:46<00:00, 542.22it/s] 


valid_parallel_last_group_wrap function took 79.110 s


53216

In [11]:
basic_feature = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'content_id']
features = basic_feature + add_feature
# features = [col for col in features if col not in ['wrong_frequent_part', 'wrong_frequent_part_count',
#                     'right_frequent_part', 'right_frequent_part_count']]
target = 'answered_correctly'
# X.fillna(0, inplace = True)
# valid.fillna(0, inplace = True)

lgb_train = lgb.Dataset(X[features].values.astype(np.float32), 
                        X[target].values.astype(np.float32), feature_name=features)
lgb_valid = lgb.Dataset(valid[features].values.astype(np.float32), 
                        valid[target].values.astype(np.float32), feature_name=features)
# del X, valid;
# del train;
gc.collect()

params = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_leaves': 63,
    'metrics':'auc',
    'n_jobs':-1
}


model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_valid],
                    verbose_eval=100,
                    num_boost_round=30000,
                    early_stopping_rounds=50
                 )

del lgb_train,lgb_valid;gc.collect()

[LightGBM] [Info] Number of positive: 26269649, number of negative: 13730351
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 15.432573 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45058
[LightGBM] [Info] Number of data points in the train set: 40000000, number of used features: 208
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.656741 -> initscore=0.648805
[LightGBM] [Info] Start training from score 0.648805
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.794921
[200]	valid_0's auc: 0.797906
[300]	valid_0's auc: 0.799349
[400]	valid_0's auc: 0.800123
[500]	valid_0's auc: 0.80066
[600]	valid_0's auc: 0.801048
[700]	valid_0's auc: 0.801319
[800]	valid_0's auc: 0.80155
[900]	valid_0's auc: 0.801724
[1000]	valid_0's auc: 0.801903
[1100]	valid_0's auc: 0.802056
[1200]	valid_0's auc: 0.802177
[1300]	valid_0's auc: 0.802289
[1400]	valid_0's auc: 0.802392
[1500]	va

188

In [54]:
basic_feature = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'content_id']
features = basic_feature + add_feature
# features = [col for col in features if col not in ['wrong_frequent_part', 'wrong_frequent_part_count',
#                     'right_frequent_part', 'right_frequent_part_count']]
target = 'answered_correctly'
# X.fillna(0, inplace = True)
# valid.fillna(0, inplace = True)

lgb_train = lgb.Dataset(X[features].values.astype(np.float32), 
                        X[target].values.astype(np.float32), feature_name=features)
lgb_valid = lgb.Dataset(valid[features].values.astype(np.float32), 
                        valid[target].values.astype(np.float32), feature_name=features)
# del X, valid;
# del train;
gc.collect()

params = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_leaves': 63,
    'metrics':'auc',
    'n_jobs':-1
}


model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_valid],
                    verbose_eval=100,
                    num_boost_round=30000,
                    early_stopping_rounds=50
                 )

del lgb_train,lgb_valid;gc.collect()

[LightGBM] [Info] Number of positive: 13067459, number of negative: 6932541
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 6.954772 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45060
[LightGBM] [Info] Number of data points in the train set: 20000000, number of used features: 208
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653373 -> initscore=0.633899
[LightGBM] [Info] Start training from score 0.633899
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.794862
[200]	valid_0's auc: 0.797873
[300]	valid_0's auc: 0.79917
[400]	valid_0's auc: 0.799844
[500]	valid_0's auc: 0.800315
[600]	valid_0's auc: 0.800603
[700]	valid_0's auc: 0.80086
[800]	valid_0's auc: 0.801068
[900]	valid_0's auc: 0.801218
[1000]	valid_0's auc: 0.801335
[1100]	valid_0's auc: 0.801434
[1200]	valid_0's auc: 0.80154
[1300]	valid_0's auc: 0.801656
[1400]	valid_0's auc: 0.801763
[1500]	valid

610

In [23]:
basic_feature = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'content_id']
features = basic_feature + add_feature
# features = [col for col in features if col not in ['wrong_frequent_part', 'wrong_frequent_part_count',
#                     'right_frequent_part', 'right_frequent_part_count']]
target = 'answered_correctly'
# X.fillna(0, inplace = True)
# valid.fillna(0, inplace = True)

lgb_train = lgb.Dataset(X[features].values.astype(np.float32), 
                        X[target].values.astype(np.float32), feature_name=features)
lgb_valid = lgb.Dataset(valid[features].values.astype(np.float32), 
                        valid[target].values.astype(np.float32), feature_name=features)
# del X, valid;
# del train;
gc.collect()

params = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_leaves': 63,
    'metrics':'auc',
    'n_jobs':-1
}


model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_valid],
                    verbose_eval=100,
                    num_boost_round=30000,
                    early_stopping_rounds=50
                 )

del lgb_train,lgb_valid;gc.collect()

[LightGBM] [Info] Number of positive: 13067459, number of negative: 6932541
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 6.836194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 44550
[LightGBM] [Info] Number of data points in the train set: 20000000, number of used features: 206
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653373 -> initscore=0.633899
[LightGBM] [Info] Start training from score 0.633899
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.794773
[200]	valid_0's auc: 0.797672
[300]	valid_0's auc: 0.798981
[400]	valid_0's auc: 0.799653
[500]	valid_0's auc: 0.800148
[600]	valid_0's auc: 0.800485
[700]	valid_0's auc: 0.800737
[800]	valid_0's auc: 0.800934
[900]	valid_0's auc: 0.801104
[1000]	valid_0's auc: 0.801264
[1100]	valid_0's auc: 0.801387
[1200]	valid_0's auc: 0.801489
[1300]	valid_0's auc: 0.801555
[1400]	valid_0's auc: 0.80164
[1500]	val

602

In [11]:
basic_feature = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'content_id']
features = basic_feature + add_feature
# features = [col for col in features if col not in ['wrong_frequent_part', 'wrong_frequent_part_count',
#                     'right_frequent_part', 'right_frequent_part_count']]
target = 'answered_correctly'
# X.fillna(0, inplace = True)
# valid.fillna(0, inplace = True)

lgb_train = lgb.Dataset(X[features].values.astype(np.float32), 
                        X[target].values.astype(np.float32), feature_name=features)
lgb_valid = lgb.Dataset(valid[features].values.astype(np.float32), 
                        valid[target].values.astype(np.float32), feature_name=features)
# del X, valid;
# del train;
gc.collect()

params = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_leaves': 63,
    'metrics':'auc',
    'n_jobs':-1
}


model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_valid],
                    verbose_eval=100,
                    num_boost_round=30000,
                    early_stopping_rounds=50
                 )

del lgb_train,lgb_valid;gc.collect()

[LightGBM] [Info] Number of positive: 13067459, number of negative: 6932541
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 6.809473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43275
[LightGBM] [Info] Number of data points in the train set: 20000000, number of used features: 201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653373 -> initscore=0.633899
[LightGBM] [Info] Start training from score 0.633899
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.794571
[200]	valid_0's auc: 0.79753
[300]	valid_0's auc: 0.798824
[400]	valid_0's auc: 0.799528
[500]	valid_0's auc: 0.799948
[600]	valid_0's auc: 0.800284
[700]	valid_0's auc: 0.800523
[800]	valid_0's auc: 0.800703
[900]	valid_0's auc: 0.800934
[1000]	valid_0's auc: 0.801054
[1100]	valid_0's auc: 0.801171
[1200]	valid_0's auc: 0.801245
[1300]	valid_0's auc: 0.801382
[1400]	valid_0's auc: 0.801473
[1500]	val

474

In [31]:
# X[['div_ratio1', 'div_ratio2', 'div_ratio3'] + \
#  [f'new_Feat{i}' for i in range(4)] + \
#  ['part_time_wrong_div', 'part_time_right_div'] +\
#  ['diff_lag_median_div', 'diff_item_median_div', 'diff_time_median_div'] +\
# ['diff_item_mean_div', 'diff_task_set_mean_div', 'diff_timestamp_mean_div'] +\
#  ['last_20_frequent_answer', 'last_20_frequent_answer_count',
#                     'last_20_frequent_answer_mean', 
#   'last_20_frequent_answer_sum']].astype(np.float32).to_feather("changed_feature.feather")

In [ ]:
# valid[['div_ratio1', 'div_ratio2', 'div_ratio3'] + \
#  [f'new_Feat{i}' for i in range(4)] + \
#  ['part_time_wrong_div', 'part_time_right_div'] +\
#  ['diff_lag_median_div', 'diff_item_median_div', 'diff_time_median_div'] +\
# ['diff_item_mean_div', 'diff_task_set_mean_div', 'diff_timestamp_mean_div'] +\
#  ['last_20_frequent_answer', 'last_20_frequent_answer_count',
#                     'last_20_frequent_answer_mean', 
#   'last_20_frequent_answer_sum']].astype(np.float32).to_feather("vaild_changed_feature.feather")

In [6]:
import pickle
with open('./model_fusion_last_v2_4000wdata.pkl', 'rb') as fin:
    model = pickle.load(fin)


In [7]:
# @timeit
def get_answer_dict(train):
    item_answer = train.groupby(['content_id', 'user_answer'])['user_answer'].agg('count').unstack()
    item_answer.fillna(0, inplace = True)
    item_answer = item_answer.to_numpy()
    total_sum = np.sum(item_answer, axis = 1)
    for i in range(4):
        item_answer[:, i] = item_answer[:, i]/total_sum
    return item_answer

content_emb_dict, content_emb_cols = content_emb_dict_init_(train)

questions = pd.read_csv('./questions.csv')
item_answer_dict = get_answer_dict(train)

answer_dict_init_ function took 6.400 s
content_encoder1 function took 27.600 s
content_encoder2 function took 11.390 s
read_enc_data function took 0.070 s
content_emb_dict_init_ function took 45.460 s


In [39]:
sys.path.append(code_path + 'beta_fusion_test_script')
from test_user_feature import initial_dict
from test_state_feature import initial_attempt, dict_trans
from test_content_encoding import content_emb_dict_init_
# from test_part_feature import part_dict_init
from test_bundle_feature import bundle_dict_init


user_dict, item_dict = initial_dict(train)


initial_dict function took 6.360 s


In [40]:
from feat_group5 import feat5_group,roll_init
from test_rolling_feature import rolling_feat_group

prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()

group5, g5_idx = feat5_group(X)
rolling_gp = roll_init(group5, 50)


roll_keep = ['user_id', 'timestamp', 'content_id', 'answered_correctly',
             'prior_question_elapsed_time', 'item_mean', 'task_set_distance', 'part', 'bundle_id', 
             'user_answer', 'correct_answer']
group, roll_idx, name_dict = rolling_feat_group(X, roll_keep)
whole_group_roll = roll_init(group, 20000)
del group, roll_idx;gc.collect()
del group5;
# del X;gc.collect()

feat5_group function took 39.010 s


In [41]:
from test_user_feature import test_user
from test_state_feature import state_feature
from test_item_feature import test_item
from test_user_feature import update_user_dict
from test_content_encoding import initial_embedding_feature
from test_feat5_group_feature import feat5_group, test_feats5_wrap
from test_rolling_feature import test_rolling_feature_wrapper
from test_part_feature import test_part_feature
from test_bundle_feature import test_bundle_feature
from test_global_group_feature import test_last_group_feature, get_last_name

######
from test_part_feature import update_part_dict
from test_bundle_feature import update_bundle_dict
from test_feat5_group_feature import rolling_df_update

In [42]:

previous_test_df = None
# del train;gc.collect()

ans_map_dict = questions.set_index(['question_id'])['correct_answer'].to_dict()
roll_keep = ['user_id', 'timestamp', 'content_id', 'answered_correctly',
             'prior_question_elapsed_time', 'item_mean', 'task_set_distance', 'part', 'bundle_id', 
             'user_answer', 'correct_answer']

pbar = tqdm(total=valid.shape[0])

cnt = 0

for (current_test, current_prediction_df) in iter_test:
    if previous_test_df is not None:
        tmp_array = np.array(eval(current_test["prior_group_answers_correct"].iloc[0]))
        tmp_array = tmp_array[tmp_array != -1]

        tmp_ans = np.array(eval(current_test["prior_group_responses"].iloc[0]))
        tmp_ans = tmp_ans[tmp_ans != -1]
        
        previous_test_df['answered_correctly'] = tmp_array
        previous_test_df['user_answer'] = tmp_ans
        user_dict = update_user_dict(user_dict, previous_test_df)
        
        test_group, test_idx = feat5_group(previous_test_df)
        rolling_gp = rolling_df_update(rolling_gp, test_group)
        
        test_roll_gp, test_roll_idx, name_dict = rolling_feat_group(previous_test_df, roll_keep)
        whole_group_roll = rolling_df_update(whole_group_roll, test_roll_gp)
        
    
    current_test = current_test[current_test.content_type_id == 0].reset_index(drop = True)

    current_test['answered_correctly'] = np.nan
    current_test['user_answer'] = np.nan
    test_ds1 = test_user(current_test, user_dict)
    test_ds3 = test_item(current_test, item_dict)
    current_test[test_ds1.columns] = test_ds1
    current_test[test_ds3.columns] = test_ds3
    
    test_content_emb_ds = initial_embedding_feature(current_test, content_emb_dict)
    current_test[content_emb_cols] = pd.DataFrame(test_content_emb_ds, index = current_test.index)
    
    test_ds5, col_name = test_feats5_wrap(current_test, rolling_gp)
    current_test[col_name] = pd.DataFrame(test_ds5, index = current_test.index)
    
    test_roll_ds, rolling_name = test_rolling_feature_wrapper(current_test, whole_group_roll, roll_keep)
    current_test[rolling_name] = pd.DataFrame(test_roll_ds, index = current_test.index)


    
    test_gp, valid_idx, name_dict = rolling_feat_group(current_test, roll_keep)
    test_last_group_ds = test_last_group_feature(test_gp, whole_group_roll, name_dict, 
                                                 valid_idx, item_answer_dict)
    last_group_name = get_last_name()
    current_test[last_group_name] = pd.DataFrame(test_last_group_ds, index=current_test.index)
    current_test['prior_question_elapsed_time'] = current_test.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
    current_test['prior_question_had_explanation'] = current_test.prior_question_had_explanation.fillna(False).astype('int8')

    features = model.feature_name()
    current_test['answered_correctly'] = model.predict(current_test[features])
    
    set_predict(current_test.loc[:,['row_id', 'answered_correctly']])
    pbar.update(len(current_test))
    previous_test_df = current_test.copy()

#     break

In [ ]:
validaten_flg = True
if validaten_flg:
    #validation score
    y_true = valid[valid.content_type_id == 0].answered_correctly
    y_pred = pd.concat(predicted).answered_correctly
    print('validation auc:',roc_auc_score(y_true, y_pred))
    
# validation auc: 0.7891487195971285

validation auc: 0.8018066807922463


 98%|█████████▊| 196158/200000 [1:04:01<02:25, 26.39it/s]

In [44]:
validaten_flg = True
if validaten_flg:
    #validation score
    y_true = valid[valid.content_type_id == 0].answered_correctly
    y_pred = pd.concat(predicted).answered_correctly
    print('validation auc:',roc_auc_score(y_true, y_pred))
    
# validation auc: 0.7891487195971285

validation auc: 0.8036493507292498


In [45]:
len(["user_id", "content_id", "part", "task_container_id", 
            "prior_question_elapsed_time", "timestamp","user_mean", 
            "answered_correctly"])

8

In [38]:
validaten_flg = True
valid = pd.read_pickle(valid_pickle)[:100000]
if validaten_flg:
    iter_test = Iter_Valid(valid,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

In [37]:
class Iter_Valid(object):
    def __init__(self, df, max_user=1000):
        df = df.reset_index(drop=True)
        self.df = df
        self.user_answer = df['user_answer'].astype(str).values
        self.answered_correctly = df['answered_correctly'].astype(str).values
        df['prior_group_responses'] = "[]"
        df['prior_group_answers_correct'] = "[]"
        self.sample_df = df[df['content_type_id'] == 0][['row_id']]
        self.sample_df['answered_correctly'] = 0
        self.len = len(df)
        self.user_id = df.user_id.values
        self.task_container_id = df.task_container_id.values
        self.content_type_id = df.content_type_id.values
        self.max_user = max_user
        self.current = 0
        self.pre_user_answer_list = []
        self.pre_answered_correctly_list = []

    def __iter__(self):
        return self
    
    def fix_df(self, user_answer_list, answered_correctly_list, pre_start):
        df= self.df[pre_start:self.current].copy()
        sample_df = self.sample_df[pre_start:self.current].copy()
        df.loc[pre_start,'prior_group_responses'] = '[' + ",".join(self.pre_user_answer_list) + ']'
        df.loc[pre_start,'prior_group_answers_correct'] = '[' + ",".join(self.pre_answered_correctly_list) + ']'
        self.pre_user_answer_list = user_answer_list
        self.pre_answered_correctly_list = answered_correctly_list
        return df, sample_df

    def __next__(self):
        added_user = set()
        pre_start = self.current
        pre_added_user = -1
        pre_task_container_id = -1

        user_answer_list = []
        answered_correctly_list = []
        while self.current < self.len:
            crr_user_id = self.user_id[self.current]
            crr_task_container_id = self.task_container_id[self.current]
            crr_content_type_id = self.content_type_id[self.current]
            if crr_content_type_id == 1:
                # no more than one task_container_id of "questions" from any single user
                # so we only care for content_type_id == 0 to break loop
                user_answer_list.append(self.user_answer[self.current])
                answered_correctly_list.append(self.answered_correctly[self.current])
                self.current += 1
                continue
            if crr_user_id in added_user and ((crr_user_id != pre_added_user) or (crr_task_container_id != pre_task_container_id)):
                # known user(not prev user or differnt task container)
                return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            if len(added_user) == self.max_user:
                if  crr_user_id == pre_added_user and crr_task_container_id == pre_task_container_id:
                    user_answer_list.append(self.user_answer[self.current])
                    answered_correctly_list.append(self.answered_correctly[self.current])
                    self.current += 1
                    continue
                else:
                    return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            added_user.add(crr_user_id)
            pre_added_user = crr_user_id
            pre_task_container_id = crr_task_container_id
            user_answer_list.append(self.user_answer[self.current])
            answered_correctly_list.append(self.answered_correctly[self.current])
            self.current += 1
        if pre_start < self.current:
            return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
        else:
            raise StopIteration()

In [17]:
valid_org = valid.copy()